In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import yaml
import matplotlib.pyplot as plt
import warnings
import numpy as np
import cv2

warnings.filterwarnings("ignore")

In [ ]:
with open(r"..\config.yaml", "r") as yml_config:
    cfg = yaml.load(yml_config, Loader=yaml.FullLoader)

image_size = cfg["image"]["image_size"]
batch_size = cfg["image"]["batch_size"]

### Visualizing some of the brain MRI scans

In [ ]:
train_ds = image_dataset_from_directory(
    r"..\data\split\train",
    image_size=image_size,
    shuffle=True,
    batch_size=batch_size,
)

for images, labels in train_ds.take(1):
    plt.figure(figsize=(20, 10))
    for _ in range(18):
        ax = plt.subplot(2, 9, _ + 1)
        plt.imshow(images[_].numpy().astype("uint8"), aspect="auto")
        class_name = train_ds.class_names[labels[_]]
        plt.title(f'Class: {class_name}')
        plt.axis('off')
    plt.show()

### Resizing the dataset

In [ ]:
# from utils import resizing
SIZE = (128, 128)

type_files = ['train', 'test', 'val']
classes = ['tumor', 'healthy']
base_path = r"C:\Users\Tony\PycharmProjects\BrainCancerDetection\data\split"

files = os.listdir(r"..\data\split")

for file in type_files:
    for class_name in classes:
        for file_name in os.listdir(os.path.join(base_path,file, class_name)):
            if file_name.lower().endswith((".png", ".jpg", ".jpeg")):
                img_path = os.path.join(base_path, file, class_name, file_name)
                img = cv2.imread(img_path)
                if img is None:
                    print(f'Skipping {img_path}, could not load the image')
                    continue
                resized_img = cv2.resize(img, SIZE)
                cv2.imread(img_path, resized_img)
